<a href="https://colab.research.google.com/github/BraveheartNL/ISIMI-2021/blob/Lennart_Project/Project/segment_nodule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from os import walk
import SimpleITK
import SimpleITK as sitk

import numpy as np
import pandas as pd
from pandas import DataFrame
from medpy.io import load, save
import random
import PIL
from PIL import Image
import scipy.ndimage
import json
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline 
plt.rcParams['figure.figsize'] = [10, 10]

import utils
import time
from tqdm import tqdm

In [3]:
config = json.load(open('config.json'))
data_folder_path = config["data_folder_path"]
ct_folder = f'{data_folder_path}/luna16_nodules/nodule_patches/'
mask_folder = f'{data_folder_path}/luna16_nodules/segmentation/'
original_xray_folder = f"{data_folder_path}/original/"
xray_folder = f"{data_folder_path}/generation_data/preprocessed_images/"
lung_segment_folder = f"{data_folder_path}/generation_data/lung_segmentations/"
results_folder = f"{data_folder_path}/extended_training_data/"
evaluation_data_folder = f"{data_folder_path}/evaluation_data/"
training_data_folder = f"{data_folder_path}/preprocessed/"
metadata_folder = f"{data_folder_path}/extended_training_data/metadata/"

_, _, xray_names = next(walk(xray_folder))
_, _, ct_names = next(walk(ct_folder))
_, _, mask_names = next(walk(mask_folder))
_, _, lung_segment_names = next(walk(lung_segment_folder))
_, _, training_data_names = next(walk(training_data_folder))

ct_names = np.sort(ct_names)
mask_names = np.sort(mask_names)
xray_names = np.sort(xray_names)
lung_segment_names = np.sort(lung_segment_names)

In [4]:
samples_count = 1
exception_count = 0
in_development = True


if (in_development):
    fig, ax = plt.subplots(samples_count, 3)

gen_metadata = {
        'height':    [],
        'img_name':  [],
        'width':     [],
        'x'    :     [],
        'y'    :     [],
        'original_name': [],
        'dataset':   [],
        }


for i in tqdm(range(samples_count)):
    try:
        xray_idx = np.random.randint(0,len(xray_names))
        xray, xray_header = load(os.path.join(xray_folder, xray_names[xray_idx]))
        xray = xray.astype(float)
        spacing = xray_header.get_voxel_spacing()
        
        # pick random nodule and mask
        ct_idx = np.random.randint(0, len(ct_names))
        ct, ct_header = load(os.path.join(ct_folder, ct_names[ct_idx]))
        ct_mask, ct_mask_header = load(os.path.join(mask_folder, mask_names[ct_idx]))
        ct = utils.preprocess_nodule(ct, ct_mask, xray, spacing)
        
        
        # pick lung mask
        lung_mask, _ = load(os.path.join(lung_segment_folder, lung_segment_names[xray_idx]))
        lung_edge_brush_thickness = int(ct.shape[0]/2) if ct.shape[0] > ct.shape[1] else int(ct.shape[1]/2)
        lung_mask = utils.preprocess_lung_mask(lung_mask, lung_edge_brush_thickness)
        
        # set spacial parameters based on pre processed lung mask
        x, y = utils.get_random_location(lung_mask)
        location = (x, y)
        width = ct.shape[0] 
        height = ct.shape[1]
        
        # normalize nodule to xray range
        ct = utils.augment_nodule(ct)[0]
        
        C = 0.25
        
        # superimpose
        result_xray = utils.spherical_superimpose((x,y), ct, xray, C)
        
        # Nodule image
        if (in_development):
            patch_width = 60 if width < 60 else width
            patch_height = 60 if height < 60 else height
            
            bb = patches.Rectangle(( int(x-0.5*patch_width), int(y-0.5*patch_height)), 
                                    patch_width, patch_height, edgecolor='r',
                                    linewidth=1, facecolor='none')
            if samples_count > 1:
                ax[i, 0].imshow(ct.T, cmap='gray')
                ax[i, 1].imshow(result_xray.T, cmap='gray')
                ax[i, 1].add_patch(bb)
                ax[i, 2].imshow(result_xray[x-width:x+width, y-height:y+height].T, cmap = 'gray')
                ax[i].set_title('temp')
            else:
                print(ct_names[ct_idx])
                ax[0].imshow(ct.T, cmap='gray')
                ax[1].imshow(result_xray.T, cmap='gray')
                ax[1].add_patch(bb)
                ax[2].imshow(result_xray[x-width:x+width, y-height:y+height].T, cmap = 'gray')
        else:
            image_name = f"{time.strftime('%Y%m%d_%H%M%S')}.mha"
            save(result_xray, f"{results_folder}/images/C{str(C)}/{image_name}")
            gen_metadata['height'].append(height)
            gen_metadata['original_name'].append('-')
            gen_metadata['width'].append(width)
            gen_metadata['x'].append(x)
            gen_metadata['y'].append(y)
            gen_metadata['img_name'].append(image_name)
            gen_metadata['dataset'].append('-')
    
    except:
        exception_count+=1
        continue
    
if (not in_development):
    metadata_names = next(walk(metadata_folder))[2]
    print(metadata_folder)
    pd.DataFrame(gen_metadata,
                columns=['height', 'img_name', 'width', 'x', 'y', 'original_name', 'dataset']).to_csv(f"{results_folder}/metadata/metadata_{len(metadata_names)}.csv",index=False)
                
print("exception rate", exception_count/samples_count)

  2%|▏         | 5/310 [00:12<14:41,  2.89s/it]